# Multiple linear regression

Exploring multiple linear regression following pace with the penguins dataset on seaborn

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
# loading the penguins dataset 
penguins = sns.load_dataset("penguins", cache=False) #if true

#exploring how our data looks like 
penguins.head(10)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,Female
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,Male
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN


In [4]:
# for this model we are working with body_mass_g, bill_length_mm, sex, species columns(features) of our dataset

#extracting these features 
penguins = penguins[["body_mass_g", "bill_length_mm", "sex", "species"]]

#renaming the columns
penguins.columns = ["body_mass_g", "bill_length_mm", "gender", "species"] # changed the name of our columns

#Drop rows with missing values
# dropna() function by default removes any rows with any missing values in any of the columns.
penguins.dropna(inplace=True) #inplace - whether to modify the datafram or creating a new one

# Reseting index
"""""
The reset_index() function resets the index values for the rows in the dataframe. 
Typically, you use reset_index() after you've finished manipulating the dataset. 
By setting inplace=True, you will not create a new DataFrame object. By setting drop=True, 
you will not insert a new index column into the DataFrame object.
"""
penguins.reset_index(inplace=True, drop=True)

In [5]:
# Eximining first 5 rows of the data 
penguins.head()

,body_mass_g,bill_length_mm,gender,species
0,3750.0,39.1,Male,Adelie
1,3800.0,39.5,Female,Adelie
2,3250.0,40.3,Female,Adelie
3,3450.0,36.7,Female,Adelie
4,3650.0,39.3,Male,Adelie


### Creating a holdout sample

creating this hold out sample to better test my regression models,
1. subsetting my x and y variables 
2. importing sci-kit learn and using a function.

In [7]:
# subseting X and Y variables 
penguins_X = penguins[["bill_length_mm", "gender", "species"]]
penguins_Y = penguins[["body_mass_g"]]

In [8]:
#importing  train-test-split function from sci-kit learn
from sklearn.model_selection import train_test_split

In [9]:
# creating training data sets and holdout (testing) data sets
X_train, X_test, Y_train, Y_test = train_test_split(penguins_X, penguins_Y, test_size=0.3, random_state=42)

relationships between datasets
1. bill length (mm) and flipper length (mm)
2. bill length (mm) and body mass (g)
3. flipper length (mm) and body mass (g)

### MODEL CONSTRUCTION

we'll focus on understanding some of the variables' relationships with body mass (g). We'll use one continuous X variable, bill length (mm), and the two categorical variables, gender and species.

First, we have to write out the formula as a string. Recall that we write out the name of the y variable first, followed by the tilde (`~`), and then each of the X variables separated by a plus sign (`+`). We can use `C()` to indicate a categorical variable. This will tell the `ols()` function to one hot encode those variables in the model. Please review the previous course materials as needed to review how and why we code categorical variables for regression.

Writing the ols formula

In [11]:
#creating the ols formula
ols_formula = "body_mass_g ~ bill_length_mm + C(gender) + C(species)"

In [12]:
# importing ols() function from stats models.formulas.api
from statsmodels.formula.api import ols

After we've imported the ols() function, we can save the ols_data as a dataframe, create the ols object, fit the model, and generate summary statistics. At this point, it would make sense to double check the model assumptions about errors (homoscedasticity and normality of residuals). Please review other resources in the program as needed.

In [13]:
# creating the ols dataframe
ols_data = pd.concat([X_train, Y_train], axis =1)

#creating OLS object and fit the model
OLS = ols(formula=ols_formula, data=ols_data)
model = OLS.fit()

### Model evaluation and interpretation

Use the .summary() function to get a summary table of model results and statistics.

Once we have our summary table, we can interpret and evaluate the model. In the upper half of the table, we get several summary statistics. We'll focus on R-squared, which tells us how much variation in body mass (g) is explained by the model. An R-squared of 0.85 is fairly high, and this means that 85% of the variation in body mass (g) is explained by the model.

Turning to the lower half of the table, we get the beta coefficients estimated by the model and their corresponding 95% confidence intervals and p-values. Based on the p-value column, labeled P>|t|, we can tell that all of the X variables are statistically significant, since the p-value is less than 0.05 for every X variable.

In [14]:
#Getting model results
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            body_mass_g   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     322.6
Date:                Thu, 13 Nov 2025   Prob (F-statistic):           1.31e-92
Time:                        05:07:39   Log-Likelihood:                -1671.7
No. Observations:                 233   AIC:                             3353.
Df Residuals:                     228   BIC:                             3371.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                2032.2111    354.087      5.739      0.000    1334.510    2729.913
C(gender)[T.Male]         528.9508     55.105      9.599      0.000     420.371     637.531
C(species)[T.Chinstrap]  -285.3865    106.339     -2.684      0.008    -494.920     -75.853
C(species)[T.Gentoo]     1081.6246     94.953     11.391      0.000     894.526    1268.723
bill_length_mm             35.5505      9.493      3.745      0.000      16.845      54.256
==============================================================================
Omnibus:                        0.339   Durbin-Watson:                   1.948
Prob(Omnibus):                  0.844   Jarque-Bera (JB):                0.436
Skew:                           0.084   Prob(JB):                        0.804
Kurtosis:                       2.871   Cond. No.                         798.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We can then interpret each of the beta coefficients for each X variable.

### C(gender) - Male
Given the name of the variable, we know that the variable was encoded as `Male = 1`, `Female = 0`. This means that female penguins are the reference point. If all other variables are constant, then we would expect a male penguin's body mass to be about 528.95 grams more than a female penguin's body mass.

### C(species) - Chinstrap and Gentoo
Given the names of these two variables, we know that Adelie penguins are the reference point. So, if we compare an Adelie penguin and a Chinstrap penguin, who have the same characteristics except their species, we would expect the Chinstrap penguin to have a body mass of about 285.39 grams less than the Adelie penguin. If we compare an Adelie penguin and a Gentoo penguin, who have the same characteristics except their species, we would expect the Gentoo penguin to have a body mass of about 1,081.62 grams more than the Adelie penguin.

### Bill length (mm)
Lastly, bill length (mm) is a continuous variable, so if we compare two penguins who have the same characteristics, except one penguin's bill is 1 millimeter longer, we would expect the penguin with the longer bill to have 35.55 grams more body mass than the penguin with the shorter bill.